In [ ]:
import os  # 운영체제와 상호작용하기 위한 모듈
import cv2  # OpenCV를 사용하여 이미지 처리하기 위한 모듈
import numpy as np  # 수치 계산을 위한 모듈
import random  # 랜덤 숫자를 생성하기 위한 모듈
import torch  # PyTorch 모듈
from tqdm import tqdm  # 진행 상황을 시각적으로 보여주기 위한 모듈
from concurrent.futures import ThreadPoolExecutor  # 병렬 처리를 위한 모듈

# 입력 디렉토리와 출력 디렉토리 경로 설정
input_directory = 'your_path'
output_directory = 'your_path'
last_processed_file = 'last_preprocessing'  # 마지막 변환된 파일명, 확장자 제외

# 출력 디렉토리가 없으면 생성
def create_directory(directory_path):
    """디렉토리가 존재하지 않으면 생성"""
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def random_adjust(image):
    """이미지의 색상, 휘도, 명암을 랜덤하게 조정하는 함수"""
    hue_shift = random.randint(-30, 30) / 180.0  # 색상 변경 범위 설정
    brightness_shift = random.randint(-50, 50) / 255.0  # 휘도 변경 범위 설정
    alpha = random.uniform(0.5, 1.5)  # 명암 변경 범위 설정

    # 조정 적용
    image = torch.clamp(image + hue_shift, 0, 1)
    image = torch.clamp(image + brightness_shift, 0, 1)
    image = torch.clamp(alpha * image, 0, 1)
    return image

def apply_split_adjust(image, mask):
    """마스크를 이용하여 이미지의 일부만 조정하는 함수"""
    adjusted_image = random_adjust(image * mask.unsqueeze(-1))  # 조정된 이미지 생성
    output_image = image * (1 - mask.unsqueeze(-1))  # 원본 이미지의 나머지 부분 유지
    output_image += adjusted_image  # 조정된 이미지와 결합
    return output_image

def create_mask(shape, points, invert=False):
    """주어진 모양과 포인트로 마스크를 생성하는 함수"""
    mask = np.zeros(shape[:2], dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.array(points), 1)
    mask = torch.tensor(mask, dtype=torch.float32, device=device)
    return 1 - mask if invert else mask

def split_and_adjust_2(image):
    """이미지를 수직으로 2분할하여 한쪽 부분만 조정하는 함수"""
    _, width, _ = image.shape
    split_width = width // 2  # 너비의 절반 계산
    left_or_right = random.choice([0, 1])  # 왼쪽 또는 오른쪽 랜덤 선택

    if left_or_right == 0:
        mask = create_mask(image.shape, [[0, 0], [split_width, 0], [split_width, image.shape[0]], [0, image.shape[0]]])
    else:
        mask = create_mask(image.shape, [[split_width, 0], [width, 0], [width, image.shape[0]], [split_width, image.shape[0]]])

    return apply_split_adjust(image, mask)

def diagonal_split_and_adjust(image):
    """이미지를 대각선으로 분할하여 한쪽 부분만 조정하는 함수"""
    mask = create_mask(image.shape, [[0, 0], [image.shape[1], 0], [image.shape[1], image.shape[0]]])
    mask = mask if random.choice([True, False]) else 1 - mask  # 랜덤으로 마스크 선택
    return apply_split_adjust(image, mask)

def split_and_adjust_3(image):
    """이미지를 3분할하여 두 부분을 조정하는 함수"""
    mask_A = create_mask(image.shape, [[image.shape[1] // 2, 0], [image.shape[1], 0], [image.shape[1], image.shape[0]], [image.shape[1] // 2, image.shape[0]]])
    mask_B = create_mask(image.shape, [[0, image.shape[0] // 2], [image.shape[1] // 2, image.shape[0] // 2], [image.shape[1] // 2, image.shape[0]], [0, image.shape[0]]])

    output_image = apply_split_adjust(image, mask_A)
    output_image = apply_split_adjust(output_image, mask_B)
    return output_image

def quarter_split_and_adjust(image):
    """이미지를 4분할하여 한 부분을 조정하는 함수"""
    height, width, _ = image.shape
    split_height = height // 2
    split_width = width // 2

    parts = [
        create_mask(image.shape, [[0, 0], [split_width, 0], [split_width, split_height], [0, split_height]]),  # 좌상단
        create_mask(image.shape, [[split_width, 0], [width, 0], [width, split_height], [split_width, split_height]]),  # 우상단
        create_mask(image.shape, [[0, split_height], [split_width, split_height], [split_width, height], [0, height]]),  # 좌하단
        create_mask(image.shape, [[split_width, split_height], [width, split_height], [width, height], [split_width, height]])  # 우하단
    ]

    mask = parts[random.randint(0, 3)]
    return apply_split_adjust(image, mask)

def process_image(filename):
    """단일 이미지를 처리하고 저장하는 함수"""
    input_path = os.path.join(input_directory, filename)
    image = cv2.imread(input_path)
    
    if image is None:
        print(f"Error reading image {filename}")
        return
    
    image = torch.tensor(image, dtype=torch.float32, device=device) / 255.0

    # 다양한 조정 함수 적
    adjusted_images = [
        split_and_adjust_2(image).cpu().numpy(),
        diagonal_split_and_adjust(image).cpu().numpy(),
        split_and_adjust_3(image).cpu().numpy(),
        quarter_split_and_adjust(image).cpu().numpy()
    ]
    
    # 조정된 이미지 저장
    for i, adjusted_image in enumerate(adjusted_images):
        output_path = os.path.join(output_directory, f'{os.path.splitext(filename)[0]}_adjusted_{i}.png')
        cv2.imwrite(output_path, (adjusted_image * 255).astype(np.uint8))
        print(f"Saved {output_path}")

def process_and_save_images():
    """입력 디렉토리의 이미지를 변환하여 출력 디렉토리에 저장하는 함수"""
    create_directory(output_directory)

    files = [f for f in os.listdir(input_directory) if f.endswith('.jpg') or f.endswith('.png')]
    files.sort()

    processed_files = set(os.path.splitext(f)[0].rsplit('_', 1)[0] for f in os.listdir(output_directory) if f.endswith('.png'))
    unprocessed_files = [f for f in files if os.path.splitext(f)[0] not in processed_files]

    start_processing = False
    with ThreadPoolExecutor(max_workers=2) as executor:
        futures = []
        for filename in tqdm(unprocessed_files, desc="Processing images"):
            if not start_processing:
                if os.path.splitext(filename)[0] == last_processed_file:
                    start_processing = True
                continue
            futures.append(executor.submit(process_image, filename))
        
        for future in futures:
            future.result()

# 함수 실행
process_and_save_images()
